In [6]:
import pandas as pd
import ast

In [7]:
# https://openweathermap.org/weather-conditions - reference for the weather column
weather_df = pd.read_csv("../../csv/weather_data_v2.csv.zip")

weather_df["weather"] = weather_df["weather"].apply(lambda x: ast.literal_eval(x)[0])

weather_df["collectedAt"] = pd.to_datetime(weather_df["collectedAt"], utc=True)
weather_df["collectedAt"] = weather_df["collectedAt"].apply(lambda x: x.to_datetime64())

weather_df["systemSunrise"] = weather_df["systemSunrise"] + weather_df["timezone"]
weather_df["systemSunrise"] = pd.to_datetime(weather_df["systemSunrise"], unit="s")
weather_df["systemSunrise"] = weather_df["systemSunrise"].apply(lambda x: x.to_datetime64())

weather_df["systemSunset"] = weather_df["systemSunset"] + weather_df["timezone"]
weather_df["systemSunset"] = pd.to_datetime(weather_df["systemSunset"], unit="s")
weather_df["systemSunset"] = weather_df["systemSunset"].apply(lambda x: x.to_datetime64())

weather_df = weather_df.drop(
    columns=["base",
             "datetime",
             "timezone",
             "id",
             "name",
             "Code",
             "longitude",
             "latitude",
             "tempMin",
             "tempMax",
             "pressure",
             "windGust",
             "systemType",
             "systemId",
             "systemCountry",
             "systemSunrise",
             "systemSunset",
             "seaLevel",
             "groundLevel"])

In [8]:
weather_df_refs = weather_df[["weather", "collectedAt", "cloudsAll"]].copy()
weather_df_refs["collectedAt"] = weather_df_refs.collectedAt.dt.floor(freq="s")

weather_df_grouped = weather_df.groupby(pd.Grouper(key="collectedAt", freq="15min")).mean()
weather_df_grouped = weather_df_grouped.reset_index()
weather_df_grouped = weather_df_grouped.drop(columns=["weather", "cloudsAll"])

In [9]:
weather_df_final = pd.merge(weather_df_grouped, weather_df_refs, on="collectedAt", how="inner")
weather_df_final = weather_df_final.fillna(value=0, axis=1)

In [10]:
weather_df_final.to_csv("../../csv/weather_data_processed_v2.csv.zip", index=False, compression="zip")